In [104]:
import numpy as np
import pandas as pd
import math

## Load Dataset

In [167]:
'''
COLUMN NAME - INDEX
gross           8
movie_title    11
language       19
country        20
'''
def loadDataset(filename):
    dataset = pd.read_csv(filename, delimiter=',')
    return np.array(dataset)

In [106]:
# DEBUG
datasetTest = loadDataset('movie_metadata.csv')
datasetTest

array([['Color', 'James Cameron', 723.0, ..., 7.9, 1.78, 33000],
       ['Color', 'Gore Verbinski', 302.0, ..., 7.1, 2.35, 0],
       ['Color', 'Sam Mendes', 602.0, ..., 6.8, 2.35, 85000],
       ...,
       ['Color', 'Benjamin Roberds', 13.0, ..., 6.3, nan, 16],
       ['Color', 'Daniel Hsia', 14.0, ..., 6.3, 2.35, 660],
       ['Color', 'Jon Gunn', 43.0, ..., 6.6, 1.85, 456]], dtype=object)

## Clearing Dataset

In [168]:
# TODO: Remove line limite  
def clearingDatasetQ1(dataset):
    subset = dataset[0:100, [8, 11]]
    new_dataset = list()
    for record in subset:
        if not(math.isnan(record[0])):
            new_dataset.append(record)
    return np.array(new_dataset)

## Possible values

In [134]:
def possibleValues(dataset):
    indexRecord = 0
    titleUniqueMovies = list()
    cleanDataset = clearingDatasetQ1(dataset)
    removingReplicates = np.copy(cleanDataset)
    
    for record in cleanDataset:
        if record[1] not in titleUniqueMovies:
            titleUniqueMovies.append(record[1])
        else:
            removingReplicates = np.delete(removingReplicates, indexRecord, 0)
            indexRecord -= 1
            
        indexRecord += 1
       
    return np.array(removingReplicates)

## Queries

In [174]:
def query1(dataset):
    maxGross = np.max(dataset[:, 0], axis=0)
    return dataset[np.where(dataset[:,0] == maxGross)][0]

## Score Function

In [135]:
def cartesianProduct(attributeA, attributeB):
    return np.transpose([np.tile(attributeA, len(attributeB)),
                         np.repeat(attributeB, len(attributeA))])

In [179]:
def scoreFunctionQ1(dataset, output):
    score = list()
       
    for record in dataset:       
        if record[1] == output:
            score.append(record[0])
        else:
            score.append(0)

    return score

## Sensitivity

In [202]:
def sensitivity(dataset, output):
    lenDataset = len(dataset) 
    output = query1(dataset)
    scoreQ1 = scoreFunctionQ1(dataset, output[1])
    minimumScoreQ1 = np.max(scoreQ1)
    
    for index in range(lenDataset):
        neighborDataset = dataset[np.delete(np.array(range(lenDataset)), index),:]
        newOutput = query1(neighborDataset)
        newScoreQ1 = scoreFunctionQ1(neighborDataset, newOutput[1])
        
        maxScoreNeighbor = np.min(newScoreQ1)
        
        if minimumScoreQ1 > maxScoreNeighbor:
            minimumScoreQ1 = maxScoreNeighbor
    
    return np.max(scoreQ1) - maxScoreNeighbor

In [204]:
def test():
    grossMovies = cleanDataset[:, 0]
    titleMovies = cleanDataset[:, 1]
    cartesianProductMoviesGross = cartesianProduct(grossMovies, titleMovies)

In [205]:
def main(filename, budgets):
    dataset = loadDataset(filename)
    cleanDataset = possibleValues(datasetTest)
    
    sensitivityQ1 = sensitivity(cleanDataset, output)
    print(sensitivityQ1)

In [206]:
main('movie_metadata.csv', [])

760505847.0
